# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [47]:
# workspace analysis
import sklearn
print(np.__version__)
print(pd.__version__)
#print(sns.__version__)
print(sklearn.__version__)
from platform import python_version
print(python_version())

1.12.1
0.23.3
0.19.1
3.6.3


In [1]:
# import libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [45]:
# import libraries
import os
import re 
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline , FeatureUnion
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC

import pickle

In [57]:
# load data from database
engine = create_engine('sqlite:///ETL_Preparation.db')

df = pd.read_sql_table('ETL_Preparation', engine)

X = df['message']   # only column 'message' relevant
y = df.iloc[:,4:]

In [58]:
##REMOVE##
# Test
#print(df.head(3))
print(df.shape)
print(type(df))
df.describe()
#Y.head(3)

(26216, 39)
<class 'pandas.core.frame.DataFrame'>


,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.766478,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.423079,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# ##REMOVE##  ALREADY done in ETL
# # remove 'child_alone' column as it has only 0 (ZERO) values.
# df = df.drop('child_alone', axis = 1)

In [52]:
# ##REMOVE##  ALREADY done in ETL
# # maximum value for 'related' column is '2'. Let us investigate further...
# print(df.groupby('related').count())
# # value '2' is strange. We can replace it with '0' or '1'. In this case, I will replace it with '1'
# df['related'] = df['related'].map(lambda x: 1 if x==2 else x) 
# # check if all '2' have been replace with '1'
# df.groupby('related').count()

            id  message  original  genre  request  offer  aid_related  \
related                                                                 
0         6122     6122      3395   6122     6122   6122         6122   
1        19906    19906      6643  19906    19906  19906        19906   
2          188      188       132    188      188    188          188   

         medical_help  medical_products  search_and_rescue      ...        \
related                                                         ...         
0                6122              6122               6122      ...         
1               19906             19906              19906      ...         
2                 188               188                188      ...         

         aid_centers  other_infrastructure  weather_related  floods  storm  \
related                                                                      
0               6122                  6122             6122    6122   6122   
1             

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20094,20094,6775,20094,20094,20094,20094,20094,20094,20094,...,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094


### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [20]:
# Test tokenize
import re 
for message in X[:5]:
        tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



In [27]:
##REMOVE##  ? from same source as above
# Build a custom transformer which extracts the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [49]:
# One pipeline for each classifier

# Pipeline1 uses AdaBoost Classifier
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

# Pipeline3 uses Linear Support Vector Machines Classifier along with a custom made transformer (StartingVerbExtractor)
pipeline3 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))
    ])

# ...
pipeline4 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [59]:
# sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline_fit = pipeline4.fit(X_train, y_train)  # change pipeline number only here!

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [62]:
y_pred_train = pipeline_fit.predict(X_train)
y_pred_test = pipeline_fit.predict(X_test)

#y_pred = pipeline1.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, y_pred_train)*100)
print(" ")

# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred_test)*100)
print(" ")

# Classification report
print("--------------------------------classification report TRAIN--------------------------------")
print(classification_report(y_train, y_pred_train, target_names=y.columns.values))
print("--------------------------------classification report TEST--------------------------------")
print(classification_report(y_test, y_pred_test, target_names=y.columns.values))

The training accuracy is: 
88.6837554674
 
The test accuracy is: 
22.4595666768
 
--------------------------------classification report TRAIN--------------------------------
                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     15033
               request       1.00      0.92      0.96      3351
                 offer       1.00      0.80      0.89        93
           aid_related       1.00      0.96      0.98      8142
          medical_help       1.00      0.83      0.91      1578
      medical_products       1.00      0.83      0.91       979
     search_and_rescue       1.00      0.79      0.88       550
              security       1.00      0.76      0.86       342
              military       1.00      0.85      0.92       649
                 water       1.00      0.91      0.95      1236
                  food       1.00      0.94      0.97      2204
               shelter       1.00      0.90      0.95    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.